# Placo Arm Control

This is a test-purpose control library for walli arm. Instead of directly solving an iterative IK, it formualtes a QP program with multiple targets and constraints, then solves it rapidly with an off-the-shelf solver (eiquadprog).

The output of this program is a series of joint position commands, published through a ROS2 publisher in the format of MultiDOFCommands. You may bring up any simulator (e.g. Gazebo with ros2_control configured, or Isaac Sim with the arm imported and omnigraph setted up) to see the robot's end effector in movement. It should draw a "inf" sign in the air.

In [2]:
# install requirements
!pip install placo

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import placo
import time
import numpy as np
import rclpy
rclpy.init()

robot = placo.RobotWrapper("/home/keseterg/Documents/RoboJackets/urc_ws/src/urc-software/urc_arm/model/arm_v1/walli_arm.urdf", placo.Flags.ignore_collisions)
robot.state.q = robot.neutral_state().q

In [4]:
solver = placo.KinematicsSolver(robot)
solver.mask_fbase(True)

T_ee = robot.get_T_world_frame("end_effector")
ee_frame_task = solver.add_frame_task("end_effector", np.eye(4))
ee_frame_task.configure("frame::ee", "soft", 10.0, 1.0)
solver.add_kinetic_energy_regularization_task(1e-6)
solver.add_regularization_task(1e2)

manipulability = solver.add_manipulability_task("end_effector", "position", 1.0)
manipulability.configure("manipulability", "soft", 1e-3)

In [5]:
# ros2 related components
from rclpy.node import Node
from control_msgs.msg import MultiDOFCommand
from typing import Sequence

class PlacoCtrl(Node):
    def __init__(self, joint_names: Sequence[str]):
        super().__init__('placo_ctrl')
        self.publisher = self.create_publisher(MultiDOFCommand, '~/ctrl_jnt_pos', 10)
        self.joint_names = joint_names
        self.cmd = MultiDOFCommand()
        self.cmd.dof_names = []
        
        for jnt_name in self.joint_names:
            self.cmd.dof_names.append(f"{jnt_name}/position")

    def pub(self, joint_states: Sequence[float]):
        assert len(joint_states) == len(self.joint_names)
        
        self.cmd.values = []
        for st in joint_states:
            self.cmd.values.append(st)
        self.publisher.publish(self.cmd)
        

In [6]:
t = 0
dt = 0.01
solver.dt = dt
last_targets = []
last_target_t = 0
ctrl = PlacoCtrl(list(robot.joint_names()))

robot.state.q = robot.neutral_state().q

def loop():
    global t, last_targets, last_target_t
    t += dt

    # Updating the effector task (drawing an infinite sign ∞)
    target = [1.2, np.cos(t) * 0.3, 0.5 + np.sin(2 * t) * 0.25]
    ee_frame_task.T_world_frame[0][3] = 1.2
    ee_frame_task.T_world_frame[1][3] = np.cos(t) * 0.3
    ee_frame_task.T_world_frame[2][3] = 0.5 + np.sin(2 * t) * 0.25
    
    # Solving the IK
    res = solver.solve(True)
    print(f"Position Output is: {res}")
    ctrl.pub(res[6:])
    robot.update_kinematics()
    

    # Drawing the last 50 targets (adding one point every 100ms)
    if t - last_target_t > 0.1:
        last_target_t = t
        last_targets.append(target)
        last_targets = last_targets[-50:]

In [7]:
for i in range(2000):
    loop()
    time.sleep(solver.dt)

Position Output is: [-0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  4.25992313e-06 -2.71482412e-02
 -3.95821912e-02 -2.38059218e-02 -5.51024988e-11  2.36014343e-09
  2.37630474e-09]
Position Output is: [-0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  2.44325516e-06 -2.69905734e-02
 -4.03532803e-02 -2.44265304e-02  4.04198498e-09  2.53999141e-09
  2.54935500e-09]
Position Output is: [-0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  5.80603149e-07 -2.66835442e-02
 -4.09272482e-02 -2.49023349e-02  6.99786027e-09  2.70522532e-09
  2.70746080e-09]
Position Output is: [-0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -1.28244757e-06 -2.62222915e-02
 -4.12879898e-02 -2.52192351e-02  6.30166327e-09  2.85083251e-09
  2.84589007e-09]
Position Output is: [-0.00000000e+00  0.00000000e+00  0.00000000e+00